In [ ]:
import sys
sys.path.append("..")
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
text = "You say goodbye and I say hello."
words = text.lower().replace(".", " .").split()
words

In [ ]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [ ]:
print(word_to_id)
print(id_to_word)

In [ ]:
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

In [ ]:
def preprocess(text):
    words = text.lower().replace(".", " .").split()
    
    word_to_id = {}
    id_to_word = {}
    
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus, word_to_id, id_to_word

In [ ]:
from common.util import preprocess

In [ ]:
# text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)

In [ ]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
            
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
    
    return co_matrix

In [ ]:
def cos_similarity(x, y, eps=1e-8):
    nx = x / np.sqrt(np.sum(x ** 2) + eps)
    ny = y / np.sqrt(np.sum(y ** 2) + eps)
    return np.dot(nx, ny)

In [ ]:
from common.util import create_co_matrix, cos_similarity

In [ ]:
# text = "You say goodbye and I say hello."
# corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id["you"]]
c1 = C[word_to_id["i"]]
print(cos_similarity(c0, c1))

In [ ]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    # get query
    if query not in word_to_id:
        print(f"{query} is not found")
        return
    
    print("[query]", query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    # compute cosine similarity
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
    
    # output top 5 cosine similarity word
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(f" {id_to_word[i]}: {similarity[i]}")
        
        count += 1
        if count >= top:
            return

In [ ]:
# text = "You say goodbye and I say hello."
# corpus, word_to_id, id_to_word = preprocess(text)
# vocab_size = len(word_to_id)
# C = create_co_matrix(corpus, vocab_size)

most_similar("you", word_to_id, id_to_word, C, top=5)

In [ ]:
def ppmi(C, verbose=False, eps=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0
    
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[j] * S[i]) + eps)
            M[i, j] = max(0, pmi)
            
            if verbose:
                cnt += 1
                if cnt % (total // 100) == 0:
                    print(f"{100 * cnt / total}% done")
    return M

In [ ]:
from common.util import ppmi

In [ ]:
W = ppmi(C)

np.set_printoptions(precision=3)
print("covariance matrix")
print(C)
print("-" * 50)
print("PPMI")
print(W)

In [ ]:
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size, window_size=1)
W = ppmi(C)
U, S, V = np.linalg.svd(W)

In [ ]:
print(C[0], W[0], U[0], sep="\n")

In [ ]:
for word, word_id in word_to_id.items():
    plt.annotate(word, (U[word_id, 0], U[word_id, 1]))

plt.scatter(U[:, 0], U[:, 1], alpha=0.5)
plt.show()

In [ ]:
!python3 show_ptb.py

In [ ]:
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data("train")

print("corpus size:", len(corpus))
print("corpus[:30]:", corpus[:30])
print()
print("id_to_word[0]:", id_to_word[0])
print("id_to_word[1]:", id_to_word[1])
print("id_to_word[2]:", id_to_word[2])
print()
print("word_to_id['car']:", word_to_id["car"])
print("word_to_id['happy']:", word_to_id["happy"])
print("word_to_id['lexus']:", word_to_id["lexus"])

In [ ]:
from sklearn.utils.extmath import randomized_svd

In [ ]:
window_size = 2
wordvec_size = 100

corpus, word_to_id, id_to_word = ptb.load_data("train")
vocab_size = len(word_to_id)
print("counting co-occurrence ...")
C = create_co_matrix(corpus, vocab_size, window_size)
print("calculating PPMI ...")
W = ppmi(C, verbose=True)

print("calculating SVD ...")
U, S, V = randomized_svd(W, n_components=wordvec_size, n_iter=5, random_state=None)

In [ ]:
word_vecs = U[:, :wordvec_size]
querys = ["you", "year", "car", "toyota"]
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)